In [1]:
import json
import torch
import jsonlines
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/miniconda3/envs/FT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "output/WebQA_LoRA/20240711084950/checkpoint-20000"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.bfloat16, device_map="cuda", trust_remote_code=True
)

In [3]:
res, history = model.chat(tokenizer, query="<user>Answer the question according to the following content \n Question: Rita Coolidge sang the title song for which Bond film?\nFollowing is the content.\nRITA COOLIDGE  ALL TIME HIGH James Bond 007 OCTOPUSSY The val doonican show 1983 - YouTube\nRITA COOLIDGE  ALL TIME HIGH James Bond 007 OCTOPUSSY The val doonican show 1983\nWant to watch this again later?\nSign in to add this video to a playlist.\nNeed to report the video?\nSign in to report inappropriate content.\nRating is available when the video has been rented.\nThis feature is not available right now. Please try again later.\nPublished on Sep 17, 2012\nClip from THE VAL DOONICAN MUSIC SHOW 1983 Featuring Rita Coolidge Performing The title track to the JAMES BOND film OCTOPUSSY.\nCategory\n<assistant>", max_length=1024, top_p=0.5)
res, history

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


('OCTOPUSSY',
 [{'role': 'user',
   'content': '<user>Answer the question according to the following content \n Question: Rita Coolidge sang the title song for which Bond film?\nFollowing is the content.\nRITA COOLIDGE  ALL TIME HIGH James Bond 007 OCTOPUSSY The val doonican show 1983 - YouTube\nRITA COOLIDGE  ALL TIME HIGH James Bond 007 OCTOPUSSY The val doonican show 1983\nWant to watch this again later?\nSign in to add this video to a playlist.\nNeed to report the video?\nSign in to report inappropriate content.\nRating is available when the video has been rented.\nThis feature is not available right now. Please try again later.\nPublished on Sep 17, 2012\nClip from THE VAL DOONICAN MUSIC SHOW 1983 Featuring Rita Coolidge Performing The title track to the JAMES BOND film OCTOPUSSY.\nCategory\n<assistant>'},
  {'role': 'assistant', 'content': 'OCTOPUSSY'}])

In [4]:
test_sample_list = []
with jsonlines.open("test_data/triviaqa-rc_web-ver_test.jsonl", 'r') as reader:
    for i, obj in enumerate(reader):
        if i >= 2000:
            break
        test_sample_list.append(obj)

In [5]:
max_input_length = 8000  # 设置输入的最大长度
output_dict = {}
for sample in tqdm(test_sample_list):
    index = sample["index"]
    user_message = sample["messages"][0]["content"]
    
    # 将用户消息限制在最大输入长度内
    user_message = user_message[:max_input_length]
    
    # 使用 model.chat() 方法进行响应，同时限制生成文本的最大长度
    res, history = model.chat(tokenizer, query=f"<user>{user_message}<assistant>", max_length=max_input_length, top_p=0.5, temperature=0.8)
    output_dict[index] = res

  1%|          | 5/410 [00:01<02:06,  3.21it/s]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe0897bd580>>
Traceback (most recent call last):
  File "/root/miniconda3/envs/FT/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
output_file = 'result_data/tuned_web.json'
# 将 output_list 写入 JSON 文件
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_dict, f, ensure_ascii=False, indent=4)

In [ ]:
test_sample_list = []
with jsonlines.open("test_data/triviaqa-rc_wiki-ver_test.jsonl", 'r') as reader:
    for i, obj in enumerate(reader):
        if i >= 2000:
            break
        test_sample_list.append(obj)
print(len(test_sample_list))

In [ ]:
max_input_length = 8000  # 设置输入的最大长度
output_dict = {}
for sample in tqdm(test_sample_list):
    index = sample["index"]
    qid = index.split('--')[0]
    user_message = sample["messages"][0]["content"]
    
    # 将用户消息限制在最大输入长度内
    user_message = user_message[:max_input_length]
    
    # 使用 model.chat() 方法进行响应，同时限制生成文本的最大长度
    res, history = model.chat(tokenizer, query=f"<user>{user_message}<assistant>", max_length=max_input_length, top_p=0.5, temperature=0.8)
    output_dict[qid] = res

In [ ]:
output_file = 'result_data/tuned_wiki.json'
# 将 output_list 写入 JSON 文件
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_dict, f, ensure_ascii=False, indent=4)